In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,StratifiedKFold
#import matplotlib
#matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import seaborn as sns
#import sweetviz as sv
#from pandas_profiling import ProfileReport
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, cohen_kappa_score, \
                            log_loss, brier_score_loss, hinge_loss, classification_report,roc_auc_score,confusion_matrix
from sklearn.svm import SVC
from sklearn.feature_selection import RFECV,RFE

import lightgbm as lgb

In [9]:
# ADMET = pd.read_excel('C:/Users/TANGLINGHUI331/Desktop/华为杯/2021年D题/ADMET.xlsx')
ER = pd.read_excel('C:/Users/TANGLINGHUI331/Desktop/华为杯/2021年D题/ERα_activity.xlsx')
Molecular = pd.read_excel('C:/Users/TANGLINGHUI331/Desktop/华为杯/2021年D题/Molecular_Descriptor.xlsx') 

In [3]:
fea_columns = Molecular.columns.drop('SMILES')
fea_columns

Index(['nAcid', 'ALogP', 'ALogp2', 'AMR', 'apol', 'naAromAtom', 'nAromBond',
       'nAtom', 'nHeavyAtom', 'nH',
       ...
       'MW', 'WTPT-1', 'WTPT-2', 'WTPT-3', 'WTPT-4', 'WTPT-5', 'WPATH', 'WPOL',
       'XLogP', 'Zagreb'],
      dtype='object', length=729)

In [4]:
# 删除0大于85%的特征
def dele0(data,columns):
    fea_columns0 = []
    for i in columns:
        if (sum(data[i]==0)/data.shape[0])<0.15:
            fea_columns0.append(i)
    return fea_columns0

In [5]:
# 删除0>85%的特征后剩余197个特征
fea_columns0 = dele0(Molecular,fea_columns)
sel_Molecular0 = Molecular[fea_columns0]

In [18]:
# 得到初步筛选后的数据集，并进行划分
X = sel_Molecular0
y = ER['pIC50']
X1 = sel_Molecular0.iloc[:1500,:]
X2 = np.mat(X1)

In [11]:
# 设置遗传算法参数
pc = 0.02      # pc为变异的概率
t = 10       #遗传算法迭代的次数
n = 500        #种群的个体数,要求大于20以保证具有随机性

In [21]:
#遗传算法
def GA(d):
    population = np.zeros((n,80))      # 初始化种群
    for i in range(n):                # 定义种群的个体数为 n
        a = np.zeros(80-d)
        b = np.ones(d)                # 将选择的d维特征定义为个体c中的1
        c = np.append(a,b)
        c = (np.random.permutation(c.T)).T    # 随机生成一个d维的个体
        population[i] = c             # 初代的种群为 population，共有n个个体
        
    # 遗传算法的迭代次数为t
    fitness_change = np.zeros(t)
    for i in range(t):
        fitness = np.zeros(n)             # fitness为每一个个体的适应度值
        for j in range(n):
            fitness[j] = Jd(population[j])          # 计算每一个体的适应度值   
        population = selection(population,fitness)  # 通过概率选择产生新一代的种群
        population = crossover(population)          # 通过交叉产生新的个体
        population = mutation(population)           # 通过变异产生新个体
        fitness_change[i] = max(fitness)      #找出每一代的适应度最大的染色体的适应度值
        
        
    # 随着迭代的进行，每个个体的适应度值应该会不断增加，所以总的适应度值fitness求平均应该会变大
    
    best_fitness = max(fitness)
    best_people = population[fitness.argmax()]
    
    return best_people,best_fitness,fitness_change,population
    
    
    


#轮盘赌选择
def selection(population,fitness):
    fitness_sum = np.zeros(n)
    for i in range(n):
        if i==0:
            fitness_sum[i] = fitness[i]
        else:
            fitness_sum[i] = fitness[i] + fitness_sum[i-1]
    for i in range(n):
        fitness_sum[i] = fitness_sum[i] / sum(fitness)
    
    #选择新的种群
    population_new = np.zeros((n,80))
    for i in range(n):
        rand = np.random.uniform(0,1)
        for j in range(n):
            if j==0:
                if rand<=fitness_sum[j]:
                    population_new[i] = population[j]
            else:
                if fitness_sum[j-1]<rand and rand<=fitness_sum[j]:
                    population_new[i] = population[j]
    return population_new
                

#交叉操作
def crossover(population):
    father = population[0:250,:]
    mother = population[250:,:]
    np.random.shuffle(father)       # 将父代个体按行打乱以随机配对
    np.random.shuffle(mother)
    for i in range(250):
        father_1 = father[i]
        mother_1 = mother[i]
        one_zero = []
        zero_one = []
        for j in range(80):
            if father_1[j]==1 and mother_1[j]==0:
                one_zero.append(j)
            if father_1[j]==0 and mother_1[j]==1:
                zero_one.append(j)
        length1 = len(one_zero)
        length2 = len(zero_one)
        length = max(length1,length2)
        half_length = int(length/2)        #half_length为交叉的位数 
        for k in range(half_length):       #进行交叉操作
            p = one_zero[k]
            q = zero_one[k]
            father_1[p]=0
            mother_1[p]=1
            father_1[q]=1
            mother_1[q]=0
        father[i] = father_1               #将交叉后的个体替换原来的个体
        mother[i] = mother_1
    population = np.append(father,mother,axis=0)
    return population
                
            
    
#变异操作
def mutation(population):
    for i in range(n):
        c = np.random.uniform(0,1)
        if c<=pc:
            mutation_s = population[i]
            zero = []                           # zero存的是变异个体中第几个数为0
            one = []                            # one存的是变异个体中第几个数为1
            for j in range(80):
                if mutation_s[j]==0:
                    zero.append(j)
                else:
                    one.append(j)
            a = np.random.randint(0,len(zero))    # e是随机选择由0变为1的位置
            b = np.random.randint(0,len(one))     # f是随机选择由1变为0的位置
            e = zero[a]
            f = one[b]
            mutation_s[e] = 1
            mutation_s[f] = 0
            population[i] = mutation_s
            
    return population


#个体适应度函数 Jd(x)，x是d维特征向量(1*80维的行向量,1表示选择该特征)
def Jd(x):
    #从特征向量x中提取出相应的特征
    Feature = np.zeros(d)        #数组Feature用来存 x选择的是哪d个特征
    k = 0
    for i in range(80):
        if x[i] == 1:
            Feature[k] = i
            k+=1
    
    #将4个特征从X_train2数据集中取出重组成一个1500*d的矩阵X_train3
    X3 = np.zeros((1500,1))
    for i in range(d):
        p = Feature[i]
        p = p.astype(int)
        q = X2[:,p]
        q = q.reshape(1500,1)
        X3 = np.append(X3,q,axis=1)
    X3 = np.delete(X3,0,axis=1)
    
    #求类间离散度矩阵Sb
    X_train3_1 = X3[0:500,:]        #数据集分为三类
    X_train3_2 = X3[500:1000,:]
    X_train3_3 = X3[1000:1500,:]
    m = np.mean(X3,axis=0)       #总体均值向量
    m1 = np.mean(X_train3_1,axis=0)    #第一类的均值向量
    m2 = np.mean(X_train3_2,axis=0)    #第二类的均值向量
    m3 = np.mean(X_train3_3,axis=0)    #第二类的均值向量
    m = m.reshape(d,1)               #将均值向量转换为列向量以便于计算
    m1 = m1.reshape(d,1)
    m2 = m2.reshape(d,1)
    m3 = m3.reshape(d,1)
    Sb = ((m1 - m).dot((m1 - m).T) + (m2 - m).dot((m2 - m).T) + (m3 - m).dot((m3 - m).T))/3 #除以类别个数
    
    #求类内离散度矩阵Sw
    S1 = np.zeros((d,d))
    S2 = np.zeros((d,d))
    S3 = np.zeros((d,d))
    for i in range(500):
        S1 += (X_train3_1[i].reshape(d,1)-m1).dot((X_train3_1[i].reshape(d,1)-m1).T)
    S1 = S1/500
    for i in range(500):
        S2 += (X_train3_2[i].reshape(d,1)-m2).dot((X_train3_2[i].reshape(d,1)-m2).T)
    S2 = S2/500
    for i in range(500):
        S3 += (X_train3_3[i].reshape(d,1)-m3).dot((X_train3_3[i].reshape(d,1)-m3).T)
    S3 = S3/500
    
    Sw = (S1 + S2 + S3)/3
    
    #计算个体适应度函数 Jd(x)
    J1 = np.trace(Sb)
    J2 = np.trace(Sw)
    Jd = J1/J2
    
    return Jd
    


if __name__ == '__main__':
    
    # best_d = np.zeros(d)          # judge存的是每一个维数的最优适应度
    
    # fitness_change是遗传算法在迭代过程中适应度变化
    # best是每一维数迭代到最后的最优的适应度，用于比较
    
    for d in range(30,31):
        print("\n")
        best_people,best_fitness,fitness_change,best_population = GA(d)
        choice = np.zeros(d)
        k = 0
        print("在取%d维的时候，通过遗传算法得出的最优适应度值为：%.6f"%(d,best_fitness))
        print("选出的最优染色体为：")
        print(best_people)
        for j in range(80):
            if best_people[j] == 1:
                choice[k]=j+1
                k+=1
        print("选出的最优特征为：")
        print(choice)
    
    '''
    #画图
    x = np.arange(0,t,1)
    plt.xlabel('dimension')
    plt.ylabel('fitness')
    plt.ylim((0,50))            # y坐标的范围
    plt.plot(x,fitness_change,'b')
    '''
    
    



在取30维的时候，通过遗传算法得出的最优适应度值为：0.067893
选出的最优染色体为：
[0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0.
 0. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1.
 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 1. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 1. 0.]
选出的最优特征为：
[ 6.  8.  9. 18. 20. 21. 27. 29. 30. 31. 32. 34. 36. 42. 43. 44. 47. 48.
 49. 51. 54. 60. 62. 63. 65. 66. 68. 69. 75. 79.]


In [22]:
# 得到遗传算法选择的30个特征
choice = choice.astype(int)
ga_fea = X.columns[choice-1]
ga_fea

Index(['nAromBond', 'nHeavyAtom', 'nH', 'ATSm2', 'ATSm4', 'ATSm5', 'BCUTw-1l',
       'BCUTc-1l', 'BCUTc-1h', 'BCUTp-1l', 'BCUTp-1h', 'nBonds2', 'nBondsS2',
       'C3SP2', 'SCH-6', 'SCH-7', 'SC-3', 'SC-5', 'VC-3', 'SPC-4', 'VPC-4',
       'SP-3', 'SP-5', 'SP-6', 'VP-0', 'VP-1', 'VP-3', 'VP-4', 'ECCEN',
       'nHaaCH'],
      dtype='object')

In [23]:
# RFECV筛选特征

# 分类
rf = RandomForestRegressor(oob_score=True)
rfe = RFE(estimator=rf,n_features_to_select=30)
rfe.fit(X,y)

print("是否应该选择 %s" % rfe.support_)


是否应该选择 [False False False False False False False False False False False False
  True  True  True False False False False False False False False False
 False False False False  True  True False False False False False False
 False False False False False False False  True False False False False
 False  True False False  True False False  True False False False False
 False False False False False False False False False False False False
  True False False False False False False False False False  True False
 False False False  True False False  True False False False False False
  True False False False False  True False False False False  True  True
  True False False False False  True False False False False False False
 False False False False False False False False False False  True False
 False False  True False False False False False False False False False
 False False False False False False False False False False False False
 False  True False False False False False F

In [24]:
# 得到RFE选择的30个特征
rfe_fea = X.columns[rfe.support_]

In [25]:
# 将遗传算法与RFE选择的特征结合
sel_fea = []
for i in ga_fea:
    sel_fea.append(i)
for j in rfe_fea:
    if j not in sel_fea:
        sel_fea.append(j)
len(sel_fea)

57

In [26]:
# 得到57个特征的训练集
sel_X = X[sel_fea]

In [33]:
sel_X_cor = sel_X.corr()

In [46]:
# RFECV筛选特征

# 分类
#rf = RandomForestRegressor(oob_score=True)
rfe1 = RFE(estimator=lgbr,n_features_to_select=30)
rfe1.fit(X,y)

print("是否应该选择 %s" % rfe1.support_)


是否应该选择 [ True False False False False False False False False False False  True
 False  True  True  True False False False False False False False False
 False False False False  True  True  True  True False False False False
 False False False False False False  True False False False False False
 False False  True False False False False False False False False False
 False False False False False False False False False False False False
  True False  True False False False False False False False  True False
 False False False False False  True False False False False False False
 False  True False False  True False  True False False False  True  True
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False  True False
 False  True False False False False False False  True False False False
 False False False False False False False F

In [47]:
# 得到RFE选择的30个特征
rfe_fea1 = X.columns[rfe1.support_]

In [51]:
# 将遗传算法与RFE选择的特征结合
sel_fea1 = []
for i in ga_fea:
    sel_fea1.append(i)
for j in rfe_fea1:
    if j not in sel_fea1:
        sel_fea1.append(j)
len(sel_fea1)

53

In [54]:
# 得到57个特征的训练集
sel_X = X[sel_fea1]

In [57]:
# 随机森林得到特征重要性
rf1 = RandomForestRegressor(random_state=0,oob_score=True)
rf1.fit(sel_X,y)
importance = rf1.feature_importances_
features = sel_X.columns

# XGBoost得到特征重要性
xgr = xgb.XGBRegressor()
xgr.fit(sel_X,y)
fscore = xgr.feature_importances_

# XGBoost得到特征重要性
lgbr = lgb.LGBMRegressor()
lgbr.fit(sel_X,y)
importance1 = lgbr.feature_importances_


In [58]:
# 得到得分和排名
score = pd.DataFrame(columns = ['features','xg_score','rf_score','lgb_score'])
score['features'] = features
score['xg_score'] = fscore
score['rf_score'] = importance
score['lgb_score']= importance1
score['lgb_score'] = final_score['lgb_score']/sum(final_score['lgb_score'])
# sort_score1 = score.sort_values(by='xg_score')
# sort_score2 = score.sort_values(by='rf_score')
score['xg_rank'] = score['xg_score'].rank(ascending=False)
score['rf_rank'] = score['rf_score'].rank(ascending=False)
score['lgb_rank'] = score['lgb_score'].rank(ascending=False)
score['score'] = (score['xg_score']+score['rf_score']+score['lgb_score'])/3
score['rank'] = score['score'].rank(ascending=False)
final_score = score.sort_values(by='rank')
# final_score.to_excel('C:/Users/TANGLINGHUI331/Desktop/49个特征排序.xlsx')

In [72]:
final_score.to_excel('C:/Users/TANGLINGHUI331/Desktop/无相似.xlsx')

In [65]:
X_sel = X[sel_fea1]

In [66]:
# 计算相关性
X_cor = X_sel.corr()

In [68]:
# 得到相关性>0.2的两列
cor_list = []
for i in range(len(X_cor.columns)):
    for j in range(i+1,len(X_cor.index)):
        if X_cor.iloc[i][j]>0.9 or X_cor.iloc[i][j]<-0.9:
            cor_list.append([X_cor.index[i],X_cor.index[j]])
print(len(cor_list))

31


In [9]:
# 根据相关性，选择删除的特征列
cor_del = []
for i in cor_list:
    cor_del.append(i[1])
fea_col1 = []
for i in fea_columns0:
    if i not in cor_del:
        fea_col1.append(i)
len(fea_col1)

80

In [19]:
sel_X.to_csv('C:/Users/TANGLINGHUI331/Desktop/华为杯/2021年D题/筛选后49个特征数据.csv')

In [22]:
# 得到得分和排名
score = pd.DataFrame(columns = ['features','xg_score','rf_score'])
score['features'] = features
score['xg_score'] = fscore
score['rf_score'] = importance
sort_score1 = score.sort_values(by='xg_score')
sort_score2 = score.sort_values(by='rf_score')
score['xg_rank'] = score['xg_score'].rank(ascending=False)
score['rf_rank'] = score['rf_score'].rank(ascending=False)
score['score'] = (score['xg_score']+score['rf_score'])/2
score['rank'] = score['score'].rank(ascending=False)
final_score = score.sort_values(by='rank')
final_score.to_excel('C:/Users/TANGLINGHUI331/Desktop/49个特征排序.xlsx')

In [31]:
fin_data = Molecular[list(final_score['features'][:20].values)]
fin_data['y']=y

<ipython-input-31-deb9bdc5de66>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fin_data['y']=y


In [37]:
fin_data.to_excel('C:/Users/TANGLINGHUI331/Desktop/筛选后49个特征数据.xlsx')

In [1]:
plt.rcParams['font.sans-serif'] = ['SimHei'] # 指定默认字体  
plt.rcParams['axes.unicode_minus'] = False # 解决保存图像是负号'-'显示为方块的问题 
plt.figure(figsize=(10,8))
sns.heatmap(fin_data.corr(method='pearson'),cmap='Blues')
plt.show()

NameError: name 'plt' is not defined